In [21]:
#!/usr/bin/env python
# coding: utf-8

In[ ]:

!pip install -r requirements.txt<br>
!pip install pipreqs<br>
print(os.listdir('data_tg'))<br>
!pipreqs --force<br>
!jupyter nbconvert --to script 'Train.ipynb'<br>
!pip install netCDF4<br>
!pip install h5py<br>
!pip install pyhdf<br>
!pip install basemap<br>
!pip install pydap<br>
!pip install xarray<br>
!pip install pygrib<br>
!pip install opencv-python<br>
!pip install zstandard

In[ ]:

In [22]:
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GroupKFold
import lightgbm as lgb
import warnings
import sklearn
from IPython.display import display
import cv2
import xarray as xr
import pydap
import h5py
from pyhdf.SD import SD, SDC
from mpl_toolkits.basemap import Basemap
import matplotlib as mpl
from joblib import Parallel, delayed
import random
import secrets
import datetime
import pygrib
import time
import zstandard as zstd
import tarfile
import io
import requests
import pickle
import subprocess
import json
import os
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
import numpy as np
get_ipython().system('pip install opencv-python-headless')

In[ ]:

In[ ]:

In[ ]:

In [23]:
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.max_open_warning'] = 100

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [24]:
pd.set_option('display.max_columns', 100)

In[ ]:

In[ ]:

s3 = boto3.client('s3')<br>
sqs = boto3.client('sqs')

In[ ]:

In [25]:
zd = zstd.ZstdDecompressor()

In[ ]:

In[ ]:

In[ ]:

In [26]:
SEED = datetime.datetime.now().microsecond

In[ ]:

In [27]:
random.seed(datetime.datetime.now().microsecond)
np.random.seed(datetime.datetime.now().microsecond)
run_label = secrets.token_hex(3)

In[ ]:

In[ ]:

In[ ]:

In [28]:
dataset = 'tg'

In[ ]:

In [29]:
ASSIM = True

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [30]:
labels = pd.read_csv('data_{}/train_labels.csv'.format(dataset))
grid = pd.concat((
    pd.read_csv('data_tg/grid_metadata.csv'),
    # pd.read_csv('data_pm/grid_metadata.csv')
)).drop_duplicates().reset_index(drop=True)

In[ ]:

In [31]:
submission = pd.read_csv('data_{}/submission_format.csv'.format(dataset))
files = pd.read_csv('data_{}/{}_satellite_metadata{}.csv'.format(
                    dataset, *(('pm25', '') if dataset == 'pm'
                               else ('no2', '_0AF3h09'))))

In[ ]:

In [32]:
labels['location'] = grid.set_index(
    'grid_id')['location'].reindex(labels.grid_id).values
labels['datetime'] = pd.to_datetime(labels.datetime)

In[ ]:

In [33]:
submission['location'] = grid.set_index(
    'grid_id').location.reindex(submission.grid_id).values

In[ ]:

loc_dict = {'la': 'Los Angeles (SoCAB)', 'tpe': 'Taipei', 'dl': 'Delhi'}

In [34]:
loc_dict = {'la': 'Los Angeles (SoCAB)'}

In[ ]:

In [35]:
coords = defaultdict(list)
for city, gcity in loc_dict.items():  # ['Delhi', 'Taipei', 'LA']:
    # gcity = [c for c in grid.location.unique() if 'Los A' in c][0] if city not in grid.location.unique() else city
    grid_points = grid[grid.location == gcity]
    for e in grid_points.itertuples():
        coords[city].append((e.grid_id, *np.array([(float(p.split(' ')[0]),
                                                    float(p.split(' ')[1]))
                                                   for p in e.wkt[10:].split(', ')[:4]]).mean(axis=0).round(4).tolist()))

In[ ]:

In[ ]:

In [36]:
def cleanDict(d):
    return {
        k: cleanDict(v) for k,
        v in d.items()} if isinstance(
        d,
        defaultdict) else d

In[ ]:

In [37]:
assert sum([len(v) for c, v in coords.items()]) == grid.grid_id.nunique()
grid.grid_id.nunique()

28

In[ ]:

In [38]:
coords = cleanDict(coords)

In[ ]:

In[ ]:

In[ ]:

t = datetime.datetime(2016, 1, 1)<br>
# t = datetime.datetime(2020, 5, 26)<br>
while t < datetime.datetime.now() - datetime.timedelta(days = 1):<br>
    print(t)<br>
    for tag in ifs_tags:<br>
    # for tag in ifs_forecasts:# ifs_tags:#[:1]:<br>
        domain = 'ec.oper.an.sfc'<br>
        # domain = 'ec.oper.fc.sfc'<br>
        file =  '{}/{}/{}.{}.regn1280sc.{}.nc'.format(domain,<br>
                        datetime.datetime.strftime(t, '%Y%m'),<br>
                        domain, tag,<br>
                        datetime.datetime.strftime(t, '%Y%m%d') )<br>
        if file.split('/')[-1] not in existing_files:<br>
            sqs.send_message(<br>
                QueueUrl='https://sqs.us-east-2.amazonaws.com/815054066888/aqi-ifs',<br>
                MessageBody=json.dumps({'file': file}<br>
            ) )<br>
        # else:<br>
        #     print(file, 'exists')<br>
    t += datetime.timedelta(days = 1)<br>
    # break;

In[ ]:

In[ ]:

In[ ]:

In [39]:
fields = ['nitrogendioxide_tropospheric_column',
          'nitrogendioxide_tropospheric_column_precision',
          'air_mass_factor_troposphere',
          'air_mass_factor_total']

In[ ]:

In[ ]:

In [40]:
trop_scr_dir = 'tropomi_data/secure.txt'
trop_secure = dict([e.split('=')
                   for e in open(trop_scr_dir, 'r').read().split('\n')])

In[ ]:

In [41]:
download_dir = '/tmp/'

In [42]:
def loadFile(row):
    for i in range(1):
        try:
            values = {
                'email': trop_secure['username'],
                'passwd': trop_secure['password'],
                'action': 'login'}
            login_url = 'https://urs.earthdata.nasa.gov'
            ret = requests.post(login_url, data=values)
            if ret.status_code == 200:
                print("Login successful.")
            else:
                print("Bad Authentication")
                return None
        except Exception as e:
            print(e)
            time.sleep(i)
    zc = zstd.ZstdCompressor(level=15)
    os.makedirs(download_dir, exist_ok=True)
    filename, url, sz = [row[k]
                         for k in ['granule_id', 'us_url', 'granuleSize']]
    # print(filename, url, sz)
    file = '/tmp/' + filename
    try:
        print("Downloading", filename)
        if os.path.exists(file):
            print(f"{file} already exists. Skipping download.")
            pass
        with requests.get(url, cookies=ret.cookies,
                          allow_redirects=True, stream=True) as r:
            r.raise_for_status()
            with open(file, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024 * 1024):
                    f.write(chunk)
        print(f"Downloaded and compressed {filename} to {file}")
    except requests.RequestException as e:
        print(f"Error downloading {url}: {e}")
    return file

In[ ]:

In[ ]:

In[ ]:

In [43]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings(
    'ignore',
    category=sklearn.exceptions.ConvergenceWarning)
warnings.filterwarnings('ignore', category=sklearn.exceptions.FitFailedWarning)

In[ ]:

In[ ]:

In[ ]:

In [44]:
fields = ['nitrogendioxide_tropospheric_column',
          'nitrogendioxide_tropospheric_column_precision',
          'air_mass_factor_troposphere',
          'air_mass_factor_total']

In[ ]:

In[ ]:

In [45]:
def run(hdf, location, fine=True):
    zones = {}  # defaultdict(dict)
    sat_data = defaultdict(lambda: defaultdict(dict))
    hp = hdf['PRODUCT']
    lat = hp['latitude'][:][0]  # .values
    lon = hp['longitude'][:][0]  # .values
    for field in fields:
        v = hp[field][:][0]
        data = np.ma.masked_array(
            v, (v == v.max()) | (
                v == v.min())).clip(
            0, None)
        assert data.shape == lat.shape
        for grid_id, plon, plat in coords[location]:
            for r in ([0.07, 0.1, 0.14, 0.2, 0.3, 0.5, 1, 2]
                      if fine else [0.1, 0.25, 0.5, 1, 2, ]):
                if (grid_id, r) not in zones:
                    zones[(grid_id, r)] = (lat - plat) ** 2 + \
                        (lon - plon) ** 2 < r ** 2
                zone = zones[(grid_id, r)]
                ct = data[zone].count()
                m = data[zone].mean() if ct > (0 if 'fine' else 3) else np.nan
                s = data[zone].std() if ct >= 3 else np.nan
                sat_data[grid_id][field + '_mean{}'.format(r)] = m
                sat_data[grid_id][field + '_stdev{}'.format(r)] = s
                sat_data[grid_id][field + '_count{}'.format(r)] = ct
                if '2' in grid_id:  # .startswith('9'):
                    print(field, '_count{}'.format(r), ct, m, s)
    return sat_data

In[ ]:

In [46]:
def runTropomi(row, fine=True):
    assert row['product'].startswith('tropomi')

    # Presumably, loadFile is a function you've defined elsewhere to load the
    # data file
    file = loadFile(row)
    hdf = h5py.File(file, 'r')

    # Assuming run is a function you've defined that processes the hdf file
    # based on the location and 'fine' parameter
    sat_data = run(hdf, row['location'], fine)
    output = row.copy()

    # Assuming cleanDict is a function you've defined to clean or process the
    # satellite data in some way
    output['d1'] = cleanDict(sat_data)

    # Compress the serialized output
    zc = zstd.ZstdCompressor(level=15)
    o = pickle.dumps(output)
    c = zc.compress(o)

    # Save the compressed data locally
    local_dir = "path/to/your/local/directory"  # Specify your directory here
    filename = 'aqi_tropomi{}-{}.zst'.format(
        '-fine' if fine else '', os.path.basename(file))
    local_path = os.path.join(local_dir, filename)
    with open(local_path, 'wb') as f:
        f.write(c)
    os.remove(file)
    return {
        'statusCode': 200,
        'len': len(o),
        'clen': len(c),
        'body': json.dumps('Success'),
        'localPath': local_path  # Optionally return the path where the file was saved
    }

In[ ]:

In[ ]:

In[ ]:

runTropomi(rows[-4], fine = True)

In[ ]:

In[ ]:

rows[-1]<br>
In[ ]:

In [47]:
get_ipython().run_cell_magic(
    'time',
    '',
    '# Parallel(os.cpu_count() #// 2\n#             )(delayed(runTropomi)(row) for row in rows)\n')

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.48 µs


In[ ]:

In[ ]:

In[ ]:

d = pd.to_datetime(files[(files['product'] == 'misr')<br>
       & (files.location == 'la')].time_end)<br>
d = d.sort_values().reset_index(drop = True)<br>
(d.diff().dt.total_seconds() / (60*60*24)).iloc[:].plot()

In[ ]:

In[ ]:

In[ ]:

coords['LA']

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

sqs.send_message(<br>
    QueueUrl='https://sqs.us-east-2.amazonaws.com/815054066888/aqi-gfs25-extra',<br>
    MessageBody=json.dumps((2020, 1, 1, 6, 0))<br>
)

t = datetime.datetime(2016, 10, 1)<br>
while t < datetime.datetime.now():<br>
    print(t)<br>
    for hr in range(0, 24, 6):<br>
        for fwd in (0, ):<br>
            sqs.send_message(<br>
                QueueUrl='https://sqs.us-east-2.amazonaws.com/815054066888/aqi-gfs25-extra',<br>
                MessageBody=json.dumps((t.year, t.month, t.day, hr, fwd))<br>
            )<br>
    t += datetime.timedelta(days = 1)

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

[e[0] for e in pickle.loads(zd.decompress(<br>
    s3.get_object(Bucket = 'projects-v',<br>
              Key = 'aqi/gfs-5/{}'.format('gfs.0p25.2016100218.f000.grib2'))['Body'].read()<br>
)) if 'LA' in e[1]]

[e[0] for e in pickle.loads(<br>
    s3.get_object(Bucket = 'projects-v',<br>
              Key = 'aqi/gfs/{}'.format('gfs.0p25.2016100112.f000.grib2'))['Body'].read()<br>
) if 'LA' in e[1]]

[e for e in pickle.loads(<br>
    s3.get_object(Bucket = 'projects-v',<br>
              Key = 'aqi/gfs/{}'.format('gfs.0p25.2019100612.f000.grib2'))['Body'].read()<br>
) if 'metre temp' in e[0]]

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

grid.wkt

grid.sort_values('location').wkt

p = pygrib.open(# 'gfs.0p25.2020072500.f000.grib2')#'gfs.0p25.2020010100.f000.grib2')<br>
    # 'gfs.0p25.2017072500.f000.grib2'<br>
    'gfs.0p25.2017072500.f000.grib2'<br>
)

In[ ]:

In[ ]:

cities = {<br>
 'Taipei': ( (121.5, 121.5), (25.0, 25) ),<br>
 'Delhi': ( (77.0, 77.25), (28.75, 28.5) ),<br>
 'LA': ((360-118.25, 360-117.75), (34.0, 34.0) )<br>
}

In[ ]:

In [48]:
cities = {
    'LA': ((360 - 118.25, 360 - 117.75), (34.0, 34.0))
}

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [49]:
get_ipython().run_cell_magic('time', '', 'coords = defaultdict(list)\n')

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 9.54 µs


In[ ]:

for city in ['Delhi', 'Taipei', 'LA']:

In [50]:
for city in ['LA']:
    gcity = [c for c in grid.location.unique(
    ) if 'Los A' in c][0] if city not in grid.location.unique() else city
    grid_points = grid[grid.location == gcity]
    for e in grid_points.itertuples():
        coords[city].append((e.grid_id, *np.array([(float(p.split(' ')[0]),
                                                    float(p.split(' ')[1]))
                                                   for p in e.wkt[10:].split(', ')[:4]]).mean(axis=0).tolist()))

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

### Load Saved Features

In[ ]:

In [51]:
def loadFiles(file):
    zd = zstd.ZstdDecompressor()
    data = []
    with tarfile.TarFile(file, 'r') as tar:
        assim_files = tar.getnames()
        for file in assim_files:
            if len(file) > 0:
                f = tar.extractfile(file)
                data.append(pickle.loads(zd.decompress(f.read())))
    return data

In[ ]:

In[ ]:

def listFiles(prefix, bucket = 'projects-v'):<br>
    paginator = s3.get_paginator('list_objects_v2')<br>
    page_iterator = paginator.paginate(Bucket = bucket,<br>
                                       Prefix = prefix )<br>
    files = []<br>
    for page in page_iterator:<br>
        files.extend([e['Key'] for e in page['Contents']])<br>
    return files

def listFiles(prefix):<br>
    return [os.path.join('cache/', prefix[4:], f)<br>
            for f in os.listdir('cache/' + prefix[4:])]

In[ ]:

In[ ]:

In[ ]:

def loadData(file):<br>
    zd = zstd.ZstdDecompressor()<br>
    data = pickle.loads(zd.decompress(open(file, 'rb').read()))<br>
    return data

In[ ]:

In[ ]:

In[ ]:

In [52]:
def flatten(l): return [e for s in l for e in s]

In[ ]:

loc_dict = {'la': 'Los Angeles (SoCAB)', 'tpe': 'Taipei', 'dl': 'Delhi'}

In [53]:
loc_dict = {'la': 'Los Angeles (SoCAB)'}

In[ ]:

In [54]:
rloc_dict = {v: k for k, v in loc_dict.items()}

In[ ]:

In [55]:
tz_dict = grid.groupby('location').tz.first().to_dict()

In[ ]:

In [56]:
loc_tz_dict = {rloc_dict[k]: v for k, v in tz_dict.items()}
loc_tz_dict

{'la': 'Etc/GMT+8'}

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

def downloadFile(file):<br>
    s3 = boto3.client('s3')<br>
    s3.download_file('projects-v', file, 'cache/' + file[4:])

def downloadFiles(files):<br>
    for path in set([f.split('/')[1] for f in files]):<br>
        os.makedirs('cache/{}'.format(path), exist_ok = True)

    Parallel(os.cpu_count() * 3)(delayed(downloadFile)(file)<br>
                                     for file in files)

In[ ]:

In[ ]:

downloadFiles(assim_files)

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [57]:
get_ipython().run_cell_magic(
    'time', '', "all_data_assim = loadFiles('cache/assim.tar')\n")

CPU times: user 10.7 s, sys: 1.3 s, total: 12 s
Wall time: 12.1 s


In[ ]:

In [58]:
all_data_assim = flatten(all_data_assim)

In[ ]:

In [59]:
len(all_data_assim)

132960

In[ ]:

In[ ]:

In[ ]:

In [60]:
rt_dict = {}

In[ ]:

In [61]:
def process(d, e):
    t = d['time_end']
    if t not in rt_dict:
        rt = (t + 30).astype('datetime64[m]')
        rt_dict[t] = rt
    rt = rt_dict[t]
    grid_data_assim[grid_id][rt].update(e)
    grid_data_assim[grid_id][rt].update({'location': d['location']})

In[ ]:

In [62]:
get_ipython().run_cell_magic(
    'time',
    '',
    "grid_data_assim = defaultdict(lambda: defaultdict(dict))\nfor d in all_data_assim:\n    for grid_id, e in d['d1'].items():\n        process(d, e)\n")

CPU times: user 14 s, sys: 494 ms, total: 14.5 s
Wall time: 14.4 s


In[ ]:

In[ ]:

In[ ]:

In [63]:
def processAssim(grid_id, v):
    df = pd.DataFrame.from_dict(v, orient='index')

    # tz
    assert df.location.nunique() == 1
    tz = loc_tz_dict[df.location.unique()[0]]
    df.index = df.index.tz_localize('UTC').tz_convert(tz).floor('1d')
    df.index.name = 'datetime'
    df.drop(columns='location', inplace=True)
    df = df.groupby(df.index).mean()  # .sort_index()#.head(10)

    # fill
    t = pd.to_datetime(submission.datetime.min()).tz_convert(tz).floor('1d')
    tf = pd.to_datetime(submission.datetime.max()).tz_convert(tz).floor('1d')
    df_t = set(df.index)
    extra_times = []
    while t <= tf:
        if t not in df_t:
            extra_times.append(t)
        t += datetime.timedelta(days=1)
    df_extra = pd.DataFrame(
        np.nan,
        index=pd.Series(extra_times),
        columns=df.columns)
    if not df_extra.empty:
        df = pd.concat([df, df_extra]).sort_index()
    else:
        # Optionally, handle the case where df_extra is empty
        # For example, you might simply want to ensure df is sorted by index
        df = df.sort_index()

    # ewm
    n = 1
    df_ewm1 = df.ewm(span=n).mean().astype(np.float32).fillna(df.mean())
    df_ewm1.columns = [c + '_{}day'.format(n) for c in df.columns]
    n = 2
    df_ewm2 = df.ewm(span=n).mean().astype(np.float32).fillna(df.mean())
    df_ewm2.columns = [c + '_{}day'.format(n) for c in df.columns]
    n = 5
    df_ewm5 = df.ewm(span=n).mean().astype(np.float32).fillna(df.mean())
    df_ewm5.columns = [c + '_{}day'.format(n) for c in df.columns]
    raw_df = df
    df = pd.concat((df_ewm1, df_ewm2, df_ewm5,
                    ), axis=1)

    # compile
    df.index = df.index.tz_localize(None)
    df.insert(0, 'grid_id', grid_id)
    return df

In[ ]:

In [64]:
get_ipython().run_cell_magic(
    'time',
    '',
    'all_dfs = Parallel(os.cpu_count())(delayed(processAssim)(grid_id, v)\n                                   for grid_id, v in grid_data_assim.items() )\n')

CPU times: user 1min 18s, sys: 4.83 s, total: 1min 23s
Wall time: 1min 26s


In[ ]:

In [65]:
assim = pd.concat(all_dfs)
assim.index.name = 'datetime'
assim = assim.reset_index().set_index(['datetime', 'grid_id'])

In[ ]:

In [66]:
del all_data_assim, grid_data_assim, all_dfs

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [67]:
get_ipython().run_cell_magic(
    'time', '', "all_data_tropomi = loadFiles('cache/tropomi-fine.tar')\n")

CPU times: user 1.79 s, sys: 36.4 ms, total: 1.82 s
Wall time: 2.04 s


In[ ]:

In [68]:
grid_data_tropomi = defaultdict(list)
for d in all_data_tropomi:
    for grid_id, e in d['d1'].items():
        # e = blend(e)#e.copy()
        e['datetime'] = d['time_end']
        e['location'] = d['location']
        grid_data_tropomi[grid_id].append(e)
    # break;

In[ ]:

In[ ]:

In[ ]:

In [69]:
get_ipython().run_cell_magic('time', '', "all_dfs = []\nfor grid_id, v in grid_data_tropomi.items():\n    df = pd.DataFrame(v)\n\n    # tz\n    assert df.location.nunique() == 1 \n    tz = loc_tz_dict[df.location.unique()[0]]\n    df.datetime = pd.to_datetime(df.datetime).dt.tz_convert(tz).dt.floor('1d')\n    df.drop(columns = 'location', inplace = True)\n\n    # group\n    for col in [c for c in df.columns if '_mean' in c]:\n        ct = np.where(df[col].isnull(), 0, df[col.replace('_mean', '_count')] )\n        df[col.replace('_mean', '_sum')] = df[col] * ct\n        df[col.replace('_mean', '_count')] = ct\n    df = df.groupby(df.datetime).sum()#.sort_index()#.head(10)\n    \n\n    for col in [c for c in df.columns if '_mean' in c]:\n        df[col] = (df[col.replace('_mean', '_sum')] \n                                / df[col.replace('_mean', '_count')]\n                    )\n    # filter\n    df = df[[c for c in df.columns if\n             ('.' in  c) and \n             ( ( '_mean' in c ) or ('column_stdev' in c) )\n            ]]\n    \n    # fill\n    t = pd.to_datetime(submission.datetime.min()).tz_convert(tz).floor('1d')\n    tf = pd.to_datetime(submission.datetime.max()).tz_convert(tz).floor('1d')\n    df_t = set(df.index)\n    extra_times = []\n    while t <= tf:\n        if t not in df_t:\n            extra_times.append(t)\n        t += datetime.timedelta(days = 1)\n    df_extra = pd.DataFrame(np.nan, index = pd.Series(extra_times), columns = df.columns )\n    if not df_extra.empty:\n        df = pd.concat([df, df_extra]).sort_index()\n    else:\n        # Optionally, handle the case where df_extra is empty\n        # For example, you might simply want to ensure df is sorted by index\n        df = df.sort_index()\n        \n    # clip\n    sigma = 4\n    high = df.ewm(span = 100).mean() + sigma * df.ewm(span = 100).std().fillna(100000)\n    df = df.clip(0, None)\n    df = np.minimum(df, high)\n    \n    # ewm     \n    n = 1\n    df_ewm1 = df.ewm(span = n).mean().astype(np.float32).fillna(df.mean())\n    df_ewm1.columns = [c + '_{}day'.format(n) for c in df.columns]\n\n    n = 3\n    df_ewm3 = df.ewm(span = n).mean().astype(np.float32).fillna(df.mean())\n    df_ewm3.columns = [c + '_{}day'.format(n) for c in df.columns]\n    \n    raw_df = df\n    df = pd.concat((df_ewm1, df_ewm3, \n                   ), axis = 1)\n    \n    # compile\n    df.index = df.index.tz_localize(None)\n    df.insert(0, 'grid_id', grid_id)\n    all_dfs.append(df)\n")

CPU times: user 5.47 s, sys: 3.08 ms, total: 5.48 s
Wall time: 5.47 s


In[ ]:

In [70]:
tropomi = pd.concat(all_dfs)
tropomi.index.name = 'datetime'
tropomi = tropomi.reset_index().set_index(['datetime', 'grid_id'])

In[ ]:

In [71]:
del all_data_tropomi, grid_data_tropomi, all_dfs

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [72]:
get_ipython().run_cell_magic('time', '', '# downloadFiles(ifs_files)\n')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In[ ]:

In[ ]:

def loadData(f):<br>
    s3 = boto3.client('s3')<br>
    zd = zstd.ZstdDecompressor()<br>
    data = pickle.loads(zd.decompress(s3.get_object(Bucket = 'projects-v', Key = f)['Body'].read()))<br>
    return data

In[ ]:

In[ ]:

In[ ]:

ifs_files = sorted(listFiles('aqi/ifs/'));

ifs_tags = [<br>
    '128_015_aluvp', '128_134_sp', '128_136_tcw',<br>
    # '128_137_tcwv',<br>
'128_164_tcc',<br>
 '128_165_10u',<br>
'128_166_10v',<br>
 '128_167_2t',<br>
'128_168_2d',<br>
'128_206_tco3',<br>
'228_246_100u',<br>
'228_247_100v',<br>
]

ifs_files = [f for f in ifs_files # if any(z in f for z in ifs_tags)<br>
                    if '.oper.fc'  in f<br>
            ]

len(ifs_files)#[::1000]

In[ ]:

In[ ]:

In[ ]:

def loadData(f):<br>
    s3 = boto3.client('s3')<br>
    data = s3.get_object(Bucket = 'projects-v', Key = f)['Body'].read()<br>
    return data

In[ ]:

In [73]:
get_ipython().run_cell_magic(
    'time',
    '',
    '# all_data_ifs = Parallel(os.cpu_count() * 3)(delayed(loadData)(d) for d in \n#                         [f for f in ifs_files[::1] ])\n# # data = flatten(all_data)\n')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.25 µs


In[ ]:

In[ ]:

In[ ]:

In [74]:
get_ipython().run_cell_magic('time', '', "all_data_ifs = loadFiles('cache/ifs.tar')\n")

CPU times: user 29.9 s, sys: 1.32 s, total: 31.2 s
Wall time: 31.2 s


In[ ]:

In[ ]:

with tarfile.open('cache/ifs.tar', "w") as tar:<br>
    for file, data in zip(ifs_files[:], all_data_ifs[:]):<br>
        t = tarfile.TarInfo(name = file)<br>
        t.size = len(data)<br>
        b = io.BytesIO(data)<br>
        b.seek(0)<br>
        tar.addfile(<br>
            t, b)<br>
        # print(b)

In[ ]:

In [75]:
get_ipython().run_cell_magic(
    'time',
    '',
    "grid_data_ifs = defaultdict(lambda: defaultdict(dict))\nfor d in all_data_ifs:\n    for dt, v in d.items():\n        for grid_id, e in v.items():\n            # e['timezone'] = grid_tz_dict[grid_id]#d['location']\n            grid_data_ifs[grid_id][dt[0]].update(e)\n")

CPU times: user 6.3 s, sys: 0 ns, total: 6.3 s
Wall time: 6.3 s


In[ ]:

In [76]:
get_ipython().run_cell_magic('time', '', "all_dfs = []\n# def compileIfs(grid_id, v):\nfor grid_id, v in grid_data_ifs.items():\n    df = pd.DataFrame(v).T\n    df.index.name = 'datetime'\n    # df.sort_index(inplace = True)\n    df = df.reset_index()\n    df.sort_values('datetime', inplace = True)\n    \n    # break;\n    # extra_rows = []\n    # t = df.datetime.min()\n    # timestamps = set(df.datetime)\n    # t_max = df.datetime.max()\n    # while t < t_max:#max(timestamps):#df.datetime.max():\n    #     t_next = t + datetime.timedelta(seconds = 60 * 60 * 12) \n    #     if t_next not in timestamps:\n    #         if grid_id == '1X116': print(t_next)\n    #         extra = df[df.datetime == t].copy() #if not last else last.copy()\n    #         extra.datetime = t_next\n    #         extra_rows.append(extra)\n    #         # last = extra\n    #     # else:\n    #         # last = None\n    #     t = t_next;\n    # #if len(extra_rows) > 1 and grid_id == '1X116': \n    # #    print('{} extra rows'.format(len(extra_rows)))\n    # df = pd.concat((df, *extra_rows))\n    \n    df.set_index('datetime', inplace = True)    \n\n    # df = addExtraRows(df.reset_index()).set_index('datetime')    \n    df = df.ewm(span = 2 * 2).mean().astype(np.float32)\n    df.columns = [c + '_{}day'.format(2) for c in df.columns]\n\n        \n    # compile\n    df.index = df.index.tz_localize(None)\n    df.drop(columns = [c for c in df.columns if 'vapour' in c \n                       or 'mean0.05' in c \n                       or 'mean0.2' in c\n                       or 'mean1' in c\n                       or 'roughness' in c\n                       or 'albedo' in c\n                      ], inplace = True)\n    df.columns = ['ifs_'+ c for c in df.columns]\n    df.insert(0, 'grid_id', grid_id)\n    # return df;\n\n    all_dfs.append(df)\n    # break;\n    # break;\n")

CPU times: user 6.25 s, sys: 75.9 ms, total: 6.33 s
Wall time: 6.21 s


all_dfs = Parallel(#os.cpu_count() * 2<br>
                  1)(delayed(compileIfs)(grid_id, v)<br>
                           for grid_id, v in grid_data_ifs.items())

In[ ]:

In [77]:
ifs = pd.concat(all_dfs)
ifs.index.name = 'datetime'
ifs = ifs.reset_index().set_index(['datetime', 'grid_id'])

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [78]:
for i in [0, 10, -1]:
    df = all_dfs[i]
    df['grid_id'] = df['grid_id'].astype(str)
    grid['grid_id'] = grid['grid_id'].astype(str)  # Ensure grid_id is string
    try:
        location = grid.set_index('grid_id').location[df.grid_id.iloc[0]]
        print(location)
    except KeyError:
        print(f"Location for grid_id {df['grid_id'].iloc[0]} not found.")
        location = "Unknown Location"  # Default or placeholder location
    df_numeric = df.select_dtypes(include=[np.number])
    # plt.matshow(df_numeric.corr())
    # plt.colorbar()
    # plt.title(location)
    # plt.show()  # Ensure the plot is shown for each iteration
    # plt.matshow(df.corr()); plt.colorbar(); plt.title(str(location))

Los Angeles (SoCAB)
Los Angeles (SoCAB)
Los Angeles (SoCAB)


In[ ]:

In [79]:
df.columns[1::6]

Index(['ifs_downward_uv_radiation_at_the_surface_mean0.1_2day',
       'ifs_surface_pressure_mean2_2day',
       'ifs_surface_sensible_heat_flux_mean0.1_2day',
       'ifs_surface_latent_heat_flux_mean2_2day',
       'ifs_10_metre_u_wind_component_mean0.1_2day',
       'ifs_10_metre_v_wind_component_mean2_2day',
       'ifs_2_metre_dewpoint_temperature_mean0.1_2day',
       'ifs_surface_solar_radiation_downwards_mean2_2day',
       'ifs_surface_net_solar_radiation_mean0.1_2day',
       'ifs_surface_net_thermal_radiation_mean2_2day',
       'ifs_total_column_ozone_mean0.1_2day',
       'ifs_total_precipitation_mean2_2day',
       'ifs_100_metre_v_wind_component_mean0.1_2day'],
      dtype='object')

In[ ]:

In [80]:
del all_data_ifs, grid_data_ifs, all_dfs

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

gfs_files = listFiles('aqi/gfs-5/')

len(gfs_files)

with tarfile.open('cache/gfs-5.tar', "w") as tar:<br>
    tar.add('cache/gfs-5/', arcname=os.path.sep)

In[ ]:

In [81]:
get_ipython().run_cell_magic(
    'time',
    '',
    "# all_data_gfs = Parallel(os.cpu_count() * 3)(delayed(loadData)(d) for d in \n#                         [f for f in gfs_files if 'f000' in f])\n# all_data_gfs = flatten(all_data_gfs)\n")

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [82]:
get_ipython().run_cell_magic(
    'time',
    '',
    "all_data_gfs = loadFiles('cache/gfs-5.tar')\nall_data_gfs = flatten(all_data_gfs)\n")

CPU times: user 1.55 s, sys: 32 ms, total: 1.59 s
Wall time: 1.59 s


In[ ]:

In [83]:
len(all_data_gfs)

53184

In[ ]:

In[ ]:

In[ ]:

In [84]:
get_ipython().run_cell_magic(
    'time',
    '',
    "points = defaultdict(lambda: defaultdict(list))\nfor e in all_data_gfs:# [e for e in h if e[1] == city]:\n    t = datetime.datetime.strptime(e[0][-12:-2], '%Y%m%d%H')\n    n = ':'.join(e[0].split(':')[1:2])\n    city = e[1]\n    \n    arr = e[3] \n    arr2 = cv2.resize(arr, None, fx = 5, fy = 5,  )\n    \n    for grid_id, x, y in coords[city]:\n        v = arr2[ int(round(( e[2][0][1] - y ) / 0.05 + 2)), \n            int(round( ( (x % 360) - e[2][1][0] )/0.05 + 2 ) ) ]\n        points[grid_id][n].append((t, v, e[-1]))\n")

CPU times: user 7.75 s, sys: 111 ms, total: 7.86 s
Wall time: 7.86 s


In[ ]:

In [85]:
def addExtraRows(df):
    extra_rows = []
    t = df.datetime.min()
    timestamps = set(df.datetime)
    t_max = df.datetime.max()
    while t < t_max:  # max(timestamps):#df.datetime.max():
        t_next = t + datetime.timedelta(seconds=60 * 60 * 6)
        if t_next not in timestamps:
            extra = df[df.datetime == t].copy() if not last else last.copy()
            extra.datetime = t_next
            extra_rows.append(extra)
            last = extra
        else:
            last = None
        t = t_next
    if len(extra_rows) > 1:
        print('{} extra rows'.format(len(extra_rows)))
    return pd.concat((df, *extra_rows))

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [86]:
all_dfs = []
for grid_point, grid_data in points.items():
    dfs = []
    for label, d in grid_data.items():
        if 'Volumetric soil moisture content' in label:
            continue
        df = pd.DataFrame(
            d,
            columns=[
                'datetime',
                label +
                '_local',
                label +
                '_city'])
        df.sort_values('datetime', inplace=True)
        dfs.append(df.set_index('datetime'))
    df = pd.concat(dfs, axis=1)
    df = addExtraRows(df.reset_index()).set_index('datetime')
    df = df.ewm(span=4 * 3).mean().astype(np.float32)
    df.columns = [c + '_{}day'.format(3) for c in df.columns]
    df['grid_id'] = grid_point
    all_dfs.append(df)
gfs = pd.concat(all_dfs)
gfs = gfs.reset_index().set_index(['datetime', 'grid_id'])
#
# print(df.corr().iloc[0, 1], label)
# break;

In[ ]:

In [87]:
del all_data_gfs, points, all_dfs

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

### Compile

In[ ]:

In [88]:
labels['dayofweek'] = labels.datetime.dt.dayofweek.astype(np.int32)

In[ ]:

In [89]:
labels['dayinyear'] = labels.datetime.dt.dayofyear.astype(np.int32)

In[ ]:

In [90]:
labels.head()

,datetime,grid_id,value,location,dayofweek,dayinyear
0,2019-01-01 08:00:00+00:00,3A3IE,8.695000,Los Angeles (SoCAB),1,1
1,2019-01-01 08:00:00+00:00,3S31A,10.496667,Los Angeles (SoCAB),1,1
2,2019-01-01 08:00:00+00:00,7II4T,37.208333,Los Angeles (SoCAB),1,1
3,2019-01-01 08:00:00+00:00,8BOQH,9.791667,Los Angeles (SoCAB),1,1
4,2019-01-01 08:00:00+00:00,A2FBI,4.308333,Los Angeles (SoCAB),1,1


In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [91]:
def addGFS(labels):
    ldo = pd.to_datetime(labels.datetime).dt.floor('6h').dt.tz_localize(None)
    df = pd.concat((labels,
                    * [2 / 3 * gfs.reindex([ldo
                                            + datetime.timedelta(seconds=60 * 60 * 12),
                                            labels.grid_id]).reset_index(drop=True)
                        + 1 / 3 * gfs.reindex([ldo
                                               + datetime.timedelta(seconds=60 * 60 * 18),
                                               labels.grid_id]).reset_index(drop=True)
                       # for k, v in gfs_ewms.items()
                       ],
                    ), axis=1)
    return df

In[ ]:

In[ ]:

gfs.groupby(['datetime', 'grid_id']).nunique().mean(axis = 1).sort_values()[-50:]

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [92]:
def ifsoffset12(c): return c.replace('ifs', 'ifs12')
def ifsoffset0(c): return c.replace('ifs', 'ifs0')

In[ ]:

In [93]:
def addIFS(labels):
    ldo = pd.to_datetime(labels.datetime).dt.floor('12h').dt.tz_localize(None)
    df = pd.concat((labels,
                    * [ifs.reindex([ldo
                                    + datetime.timedelta(seconds=60 * 60 * 12),
                                    labels.grid_id]).reset_index(drop=True).rename(
                        columns=ifsoffset12),
                        ifs.reindex([ldo                            # + datetime.timedelta(seconds = 60 * 60 * 0)
                                     ,
                                     labels.grid_id]).reset_index(drop=True).rename(
                        columns=ifsoffset0)
                       # for k, v in gfs_ewms.items()
                       ],
                    ), axis=1)
    return df

In[ ]:

In[ ]:

In[ ]:

In [94]:
def addSat(labels, sat):
    tza = labels.datetime.copy()
    for location in labels.location.unique():
        t = pd.to_datetime(labels.datetime).dt.tz_convert(tz_dict[location])
        t = t.dt.floor('1d').dt.tz_localize(None)
        tza = np.where(labels.location == location, t, tza)
    tza = pd.to_datetime(pd.Series(tza, index=labels.index))
    df = pd.concat((labels,
                    sat.reindex([tza,
                                 labels.grid_id]).reset_index(drop=True)
                    ), axis=1)
    return df

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [95]:
all_data = addGFS(labels)
all_data = addIFS(all_data)
all_data = addSat(all_data, 'maiac' if dataset == 'pm' else tropomi)
if ASSIM:
    all_data = addSat(all_data, assim)
# all_data = addSat(all_data, misr)

In[ ]:

In [96]:
all_data.tail()

,datetime,grid_id,value,location,dayofweek,dayinyear,Wind speed (gust)_local_3day,Wind speed (gust)_city_3day,Surface pressure_local_3day,Surface pressure_city_3day,2 metre temperature_local_3day,2 metre temperature_city_3day,2 metre specific humidity_local_3day,2 metre specific humidity_city_3day,2 metre relative humidity_local_3day,2 metre relative humidity_city_3day,Apparent temperature_local_3day,Apparent temperature_city_3day,10 metre U wind component_local_3day,10 metre U wind component_city_3day,10 metre V wind component_local_3day,10 metre V wind component_city_3day,Cloud water_local_3day,Cloud water_city_3day,Relative humidity_local_3day,Relative humidity_city_3day,Total ozone_local_3day,Total ozone_city_3day,ifs12_downward_uv_radiation_at_the_surface_mean0.1_2day,ifs12_downward_uv_radiation_at_the_surface_mean0.5_2day,ifs12_downward_uv_radiation_at_the_surface_mean2_2day,ifs12_downward_uv_radiation_at_the_surface_mean5_2day,ifs12_surface_pressure_mean0.1_2day,ifs12_surface_pressure_mean0.5_2day,ifs12_surface_pressure_mean2_2day,ifs12_surface_pressure_mean5_2day,ifs12_total_column_water_mean0.1_2day,ifs12_total_column_water_mean0.5_2day,ifs12_total_column_water_mean2_2day,ifs12_total_column_water_mean5_2day,ifs12_surface_sensible_heat_flux_mean0.1_2day,ifs12_surface_sensible_heat_flux_mean0.5_2day,ifs12_surface_sensible_heat_flux_mean2_2day,ifs12_surface_sensible_heat_flux_mean5_2day,ifs12_surface_latent_heat_flux_mean0.1_2day,ifs12_surface_latent_heat_flux_mean0.5_2day,ifs12_surface_latent_heat_flux_mean2_2day,ifs12_surface_latent_heat_flux_mean5_2day,ifs12_total_cloud_cover_mean0.1_2day,ifs12_total_cloud_cover_mean0.5_2day,...,co_mean0.1_2day,co_mean0.25_2day,co_mean0.5_2day,co_mean1_2day,co_mean2_2day,no2_mean0.1_2day,no2_mean0.25_2day,no2_mean0.5_2day,no2_mean1_2day,no2_mean2_2day,o3_mean0.1_2day,o3_mean0.25_2day,o3_mean0.5_2day,o3_mean1_2day,o3_mean2_2day,pm25_rh35_gcc_mean0.1_2day,pm25_rh35_gcc_mean0.25_2day,pm25_rh35_gcc_mean0.5_2day,pm25_rh35_gcc_mean1_2day,pm25_rh35_gcc_mean2_2day,so2_mean0.1_2day,so2_mean0.25_2day,so2_mean0.5_2day,so2_mean1_2day,so2_mean2_2day,co_mean0.1_5day,co_mean0.25_5day,co_mean0.5_5day,co_mean1_5day,co_mean2_5day,no2_mean0.1_5day,no2_mean0.25_5day,no2_mean0.5_5day,no2_mean1_5day,no2_mean2_5day,o3_mean0.1_5day,o3_mean0.25_5day,o3_mean0.5_5day,o3_mean1_5day,o3_mean2_5day,pm25_rh35_gcc_mean0.1_5day,pm25_rh35_gcc_mean0.25_5day,pm25_rh35_gcc_mean0.5_5day,pm25_rh35_gcc_mean1_5day,pm25_rh35_gcc_mean2_5day,so2_mean0.1_5day,so2_mean0.25_5day,so2_mean0.5_5day,so2_mean1_5day,so2_mean2_5day
17468,2020-10-31 08:00:00+00:00,WT52R,57.340000,Los Angeles (SoCAB),5,305,2.946614,1.834377,89382.109375,99771.445312,289.004700,295.521179,0.003028,0.005841,27.932375,35.447212,288.714905,295.485382,-1.452794,0.423094,-0.754372,0.516548,0.003454,0.005567,9.328480,11.56105,274.624756,278.495056,1.065268e+06,1.077449e+06,1.036963e+06,1.021704e+06,90879.773438,90035.242188,95265.109375,94720.851562,6.562494,6.550615,8.474389,9.253974,-1535972.375,-2306815.000,-1555489.875,-1324481.875,-1.673854e+06,-637893.12500,-392214.7500,-1011998.875,0.385165,0.378732,...,1.373790e-07,1.406830e-07,1.484084e-07,1.722666e-07,1.565104e-07,6.425241e-09,6.503162e-09,6.755912e-09,7.579267e-09,4.630005e-09,3.762781e-08,3.784224e-08,3.831247e-08,3.827588e-08,4.079939e-08,6.979756,7.337347,8.362208,11.036541,11.097125,1.177125e-10,1.225221e-10,1.433433e-10,1.848127e-10,2.676509e-10,1.501006e-07,1.525754e-07,1.587825e-07,1.824505e-07,1.615554e-07,6.573737e-09,6.579260e-09,6.626611e-09,7.225667e-09,4.458632e-09,3.685041e-08,3.706931e-08,3.758790e-08,3.764673e-08,4.023135e-08,9.367173,9.754875,10.936839,14.149054,13.116908,1.468811e-10,1.523360e-10,1.751985e-10,2.253660e-10,2.942155e-10
17469,2020-10-31 08:00:00+00:00,X5DKW,86.166667,Los Angeles (SoCAB),5,305,2.535365,1.834377,96147.906250,99771.445312,294.864197,295.521179,0.003873,0.005841,23.460022,35.447212,294.748108,295.485382,0.057169,0.423094,-0.521182,0.516548,0.005363,0.005567,

In[ ]:

In [97]:
all_data.head()

,datetime,grid_id,value,location,dayofweek,dayinyear,Wind speed (gust)_local_3day,Wind speed (gust)_city_3day,Surface pressure_local_3day,Surface pressure_city_3day,2 metre temperature_local_3day,2 metre temperature_city_3day,2 metre specific humidity_local_3day,2 metre specific humidity_city_3day,2 metre relative humidity_local_3day,2 metre relative humidity_city_3day,Apparent temperature_local_3day,Apparent temperature_city_3day,10 metre U wind component_local_3day,10 metre U wind component_city_3day,10 metre V wind component_local_3day,10 metre V wind component_city_3day,Cloud water_local_3day,Cloud water_city_3day,Relative humidity_local_3day,Relative humidity_city_3day,Total ozone_local_3day,Total ozone_city_3day,ifs12_downward_uv_radiation_at_the_surface_mean0.1_2day,ifs12_downward_uv_radiation_at_the_surface_mean0.5_2day,ifs12_downward_uv_radiation_at_the_surface_mean2_2day,ifs12_downward_uv_radiation_at_the_surface_mean5_2day,ifs12_surface_pressure_mean0.1_2day,ifs12_surface_pressure_mean0.5_2day,ifs12_surface_pressure_mean2_2day,ifs12_surface_pressure_mean5_2day,ifs12_total_column_water_mean0.1_2day,ifs12_total_column_water_mean0.5_2day,ifs12_total_column_water_mean2_2day,ifs12_total_column_water_mean5_2day,ifs12_surface_sensible_heat_flux_mean0.1_2day,ifs12_surface_sensible_heat_flux_mean0.5_2day,ifs12_surface_sensible_heat_flux_mean2_2day,ifs12_surface_sensible_heat_flux_mean5_2day,ifs12_surface_latent_heat_flux_mean0.1_2day,ifs12_surface_latent_heat_flux_mean0.5_2day,ifs12_surface_latent_heat_flux_mean2_2day,ifs12_surface_latent_heat_flux_mean5_2day,ifs12_total_cloud_cover_mean0.1_2day,ifs12_total_cloud_cover_mean0.5_2day,...,co_mean0.1_2day,co_mean0.25_2day,co_mean0.5_2day,co_mean1_2day,co_mean2_2day,no2_mean0.1_2day,no2_mean0.25_2day,no2_mean0.5_2day,no2_mean1_2day,no2_mean2_2day,o3_mean0.1_2day,o3_mean0.25_2day,o3_mean0.5_2day,o3_mean1_2day,o3_mean2_2day,pm25_rh35_gcc_mean0.1_2day,pm25_rh35_gcc_mean0.25_2day,pm25_rh35_gcc_mean0.5_2day,pm25_rh35_gcc_mean1_2day,pm25_rh35_gcc_mean2_2day,so2_mean0.1_2day,so2_mean0.25_2day,so2_mean0.5_2day,so2_mean1_2day,so2_mean2_2day,co_mean0.1_5day,co_mean0.25_5day,co_mean0.5_5day,co_mean1_5day,co_mean2_5day,no2_mean0.1_5day,no2_mean0.25_5day,no2_mean0.5_5day,no2_mean1_5day,no2_mean2_5day,o3_mean0.1_5day,o3_mean0.25_5day,o3_mean0.5_5day,o3_mean1_5day,o3_mean2_5day,pm25_rh35_gcc_mean0.1_5day,pm25_rh35_gcc_mean0.25_5day,pm25_rh35_gcc_mean0.5_5day,pm25_rh35_gcc_mean1_5day,pm25_rh35_gcc_mean2_5day,so2_mean0.1_5day,so2_mean0.25_5day,so2_mean0.5_5day,so2_mean1_5day,so2_mean2_5day
0,2019-01-01 08:00:00+00:00,3A3IE,8.695000,Los Angeles (SoCAB),1,1,5.413679,4.403292,92828.007812,100089.90625,281.026093,286.235413,0.002560,0.002854,35.595852,30.025583,279.274139,286.091034,-2.047880,-1.76149,-1.687304,-0.21584,0.004267,0.005275,10.903458,12.244355,329.625122,332.235718,745171.8750,739461.6875,711426.5000,690866.3750,97174.710938,94290.960938,95142.187500,95286.648438,4.466324,3.908245,4.787787,6.222759,-1149612.625,-1676964.875,-1217090.750,-1097625.125,-1.568464e+06,-6.496657e+05,-2806355.0,-3111115.75,0.044663,0.039892,...,2.645851e-07,2.654458e-07,2.435152e-07,1.861290e-07,1.489865e-07,1.072726e-08,1.084041e-08,9.688059e-09,5.729664e-09,2.602033e-09,2.922568e-08,2.909989e-08,3.022860e-08,3.413607e-08,3.754369e-08,13.694448,13.684081,12.449080,9.186067,7.114766,3.690123e-10,3.700938e-10,3.349736e-10,2.383904e-10,1.841198e-10,3.293613e-07,3.280886e-07,2.926124e-07,2.129792e-07,1.612483e-07,1.407059e-08,1.402868e-08,1.211337e-08,6.980736e-09,3.080813e-09,2.672559e-08,2.674875e-08,2.853741e-08,3.329452e-08,3.720178e-08,21.200350,21.053583,18.760553,13.533575,9.749266,4.287931e-10,4.265869e-10,3.683897e-10,2.396264e-10,1.726736e-10
1,2019-01-01 08:00:00+00:00,3S31A,10.496667,Los Angeles (SoCAB),1,1,5.968803,4.403292,100819.125000,100089.90625,286.146179,286.235413,0.003239,0.002854,34.337368,30.025583,285.988281,286.091034,-2.845850,-1.76149,-0.686538,-0.21584,0.008124,0.005275,12.777002,12.2

In[ ]:

In[ ]:

In[ ]:

In [98]:
x = all_data[[c for c in all_data.columns if c not in [
    'datetime', 'value']]].copy()

In[ ]:

'7334C' - > '7F1D1'<br>
'HANW9' - > 'WZNCR'

In [99]:
if dataset == 'tg':
    np.random.seed(SEED)
    x.loc[(x.grid_id == '7334C') & (
        np.random.random(len(x)) < 0.1), 'grid_id'] = '7F1D1'
    x.loc[(x.grid_id == 'HANW9') & (
        np.random.random(len(x)) < 0.1), 'grid_id'] = 'WZNCR'

In[ ]:

In [100]:
x.grid_id = x.grid_id.astype('category')
x.location = x.location.astype('category')

In[ ]:

In [101]:
y = all_data.value.astype(np.float32)
d = all_data.datetime

In[ ]:

In [102]:
pickle.dump(all_data, open('cache/all_data_{}.pkl'.format(dataset), 'wb'))

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [103]:
x.columns = [c.replace(':', '_') for c in x.columns]

In[ ]:

In [104]:
x.shape

(17473, 313)

In[ ]:

In[ ]:

x.fillna(method = 'ffill')

x.isnull().sum()

x.isnull().sum().sort_values()

In[ ]:

In [105]:
assert x.isnull().sum().sum() == 0

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [106]:
enet_params = {
    'alpha': [1e-2, 3e-2, 1e-1, 0.3, 1, 3, ],
    'l1_ratio': [0.01, 0.03, 0.1, 0.2, 0.5, 0.8, 0.9, ],
}

In[ ]:

In [107]:
class PurgedKFold():
    def __init__(self, n_splits=5, gap=30):
        self.n_splits = n_splits
        self.gap = gap
    def get_n_splits(self, X, y=None, groups=None): return self.n_splits
    def split(self, X, y=None, groups=None):
        groups = groups.sort_values()
        X = X.reindex(groups.index)  # sort_values(groups)
        y = y.reindex(X.index)
        X, y, groups = sklearn.utils.indexable(X, y, groups)
        indices = np.arange(len(X))
        n_splits = self.n_splits
        for i in range(n_splits):
            test = indices[i * len(X) // n_splits: (i + 1)
                           * len(X) // n_splits]  # .index
            train = indices[(groups <= groups.iloc[test].min() -
                             datetime.timedelta(days=self.gap)) | (groups >= groups.iloc[test].max() +
                                                                   datetime.timedelta(days=self.gap))]  # .index
            yield train, test

In[ ]:

In [108]:
class RepeatedPurgedKFold():
    def __init__(self, n_splits=5, n_repeats=1, gap=None):
        self.n_splits = n_splits
        self.n_repeats = n_repeats
        self.gap = gap
    def get_n_splits(self, X, y=None, groups=None):
        return self.n_splits * self.n_repeats + \
            self.n_repeats * (self.n_repeats - 1) // 2
    def split(self, X, y=None, groups=None):
        for i in range(self.n_repeats):
            for f in PurgedKFold(
                    self.n_splits + i,
                    gap=self.gap if self.gap else None).split(
                    X,
                    y,
                    groups):
                yield f

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [109]:
def posRMSE(y, y_pred):
    return mean_squared_error(
        y,
        y_pred.clip(
            0,
            None) *
        2 /
        3 +
        y_pred *
        1 /
        3) ** 0.5

In [110]:
pRMSE = make_scorer(posRMSE, greater_is_better=False)
SCORING = pRMSE

In[ ]:

In[ ]:

In[ ]:

In [111]:
COLUMN_WIPE = 0  # len(x.columns) // 10

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [112]:
def runENet(drop_cols=[], verbose=1):
    all_y_enet = []
    all_y_pred_enet = []
    enet_clfs = []
    enet_scalers = []
    for location in x.location.unique():
        if verbose > 0:
            print(location)
        x_loc = x[x.location == location].drop(columns=drop_cols)
        y_loc = y.reindex(x_loc.index)
        d_loc = d.reindex(x_loc.index)
        folds = list(
            PurgedKFold(
                4 if dataset == 'pm' else 3).split(
                x_loc,
                y_loc,
                d_loc))
        folds += [(np.arange(0, len(x_loc)), [])] * 2
        for train_fold, test_fold in folds:
            y_preds = []
            for i in range(8):
                scaler = StandardScaler()
                clf = ElasticNet(
                    max_iter=50000,
                    tol=1e-3,
                    selection='random',
                    precompute=True,
                    random_state=datetime.datetime.now().microsecond)
                model = RandomizedSearchCV(clf, enet_params,
                                           scoring=SCORING,
                                           # 'neg_root_mean_squared_error',
                                           cv=RepeatedPurgedKFold(random.randrange(3, 6),
                                                                  n_repeats=random.randrange(
                                                                      2, 4),
                                                                  gap=random.randrange(60, 120)),
                                           n_iter=random.randrange(4, 7),
                                           random_state=datetime.datetime.now().microsecond)
                # subset = train_fold#[:s].tolist() + train_fold[s +
                # l:].tolist()
                l = random.randrange(0, len(train_fold) // 10)
                s = random.randrange(0, len(train_fold) - l)
                subset = train_fold[:s].tolist() + train_fold[s + l:].tolist()
                xt = x_loc.iloc[subset, 2:].copy()
                # for c in misr.columns:
                if dataset == 'pm':
                    sample_cols = [
                        c for c in xt.columns if any(
                            z in c for z in [
                                'precision',
                                'air_mass',
                                'stdev'] + [
                                'no2',
                                'so2',
                                'co',
                                'o3',
                                'pm25_rh35_gcc',
                                'ifs'])]
                    xt[random.sample(sample_cols, k=random.randrange(
                        len(sample_cols) * 1 // 3, len(sample_cols) * 2 // 3))] = 0
                if dataset == 'tg':
                    if random.random() < 0.5:
                        xt[[c for c in xt.columns if 'precision' in c]] = 0
                    # if random.random() < 0.5: xt[[c for c in xt.columns if 'mean1' in c or 'mean2' in c]] = 0
                # xt[misr.columns] = 0
                # fj_drop =
                # if random.random() < {'Delhi': 0.5, 'Taipei': 0.2}.get(location, 0):
                #     xt[[c for c in xt.columns if 'FineMode' in c or 'Injection' in c]] = 0
                # # if random.random() < {'Delhi': 0.2, 'Taipei': 0.1}.get(location, 0):
                # #     xt[[c for c in xt.columns if 'Optical' in c]] = 0
                # if random.random() < {'Delhi': 0.85, 'Taipei': 0.5}.get(location, 0.2):
                #     xt[[c for c in xt.columns if c.startswith('ifs')]] = 0
                # xt = x_loc.iloc[subset].copy()
                xt[random.choices(xt.columns[2:], k=int(
                    round(random.random() * COLUMN_WIPE)))] = 0
                if random.random() < 0.3:
                    xt['dayinyear'] = 0
                model.fit(
                    pd.DataFrame(
                        scaler.fit_transform(xt).astype(
                            np.float32),
                        xt.index,
                        xt.columns),
                    y_loc.iloc[subset],
                    groups=d_loc.iloc[subset])
                enet_clfs.append(model.best_estimator_)
                enet_scalers.append(scaler)
                if i == 0 and verbose > 0:
                    display(
                        pd.DataFrame(
                            model.cv_results_).sort_values('rank_test_score').drop(
                            columns='params'))
                if len(test_fold) > 0:
                    y_pred = pd.Series(model.predict(
                        pd.DataFrame(scaler.transform(x_loc.iloc[test_fold, 2:]).astype(np.float32),
                                     columns=x_loc.columns[2:])),  # .clip(0, None),
                        index=y_loc.iloc[test_fold].index)
                    y_preds.append(y_pred)
            if len(test_fold) > 0:
                y_pred = pd.concat(y_preds)
                y_pred = y_pred.clip(
                    0,
                    None).groupby(
                    y_pred.index).mean().clip(
                    0,
                    None)
                y_pred = (y_pred.groupby(y_pred.index).mean())  # *2/3
                # + y_pred.groupby(y_pred.index).median() * 1/3 )
                all_y_enet.append(y_loc.iloc[test_fold])
                all_y_pred_enet.append(y_pred)
                if verbose >= 1:
                    print(location, round(np.corrcoef(
                        y_pred, all_y_enet[-1])[0, 1], 4))
                if verbose >= 3:
                    all_y_enet[-1].reset_index(
                        drop=True).ewm(span=10).mean().plot()
                    plt.plot(y_pred.reset_index(drop=True))
                    plt.title(location)
                    plt.figure()
                    print((x_loc.iloc[subset, 2:].std().clip(1, 1) *
                           model.best_estimator_.coef_).sort_values())
    return all_y_enet, all_y_pred_enet, enet_clfs, enet_scalers

In[ ]:

In [113]:
all_y_enet, all_y_pred_enet, enet_clfs, enet_scalers = runENet(verbose=2)

Los Angeles (SoCAB)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_l1_ratio,param_alpha,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,split10_test_score,split11_test_score,split12_test_score,split13_test_score,split14_test_score,split15_test_score,split16_test_score,split17_test_score,mean_test_score,std_test_score,rank_test_score
0,0.027220,0.001543,0.005918,0.000362,0.5,0.1,-12.172274,-14.641259,-10.052961,-9.059697,-14.677820,-12.132236,-14.351908,-10.412288,-9.960575,-9.397174,-15.499084,-11.834753,-14.647692,-12.008248,-10.078034,-8.838559,-10.390071,-15.980052,-12.007483,2.321617,1
2,0.028574,0.001563,0.006450,0.001138,0.8,0.1,-12.070040,-14.212795,-10.142167,-9.004812,-15.232786,-12.041973,-14.100404,-10.371619,-9.855815,-9.372439,-16.135208,-11.744478,-14.448021,-11.799708,-10.010086,-8.780326,-10.411413,-16.589699,-12.017988,2.443233,2
3,0.025030,0.001694,0.005909,0.000153,0.8,0.3,-12.864180,-13.072490,-10.121971,-9.688903,-15.487473,-12.912354,-13.279616,-10.669940,-10.269005,-9.770783,-16.243866,-12.562339,-14.035787,-11.408261,-10.075085,-9.462468,-10.513178,-16.680954,-12.173259,2.256835,3
1,0.023927,0.001836,0.006394,0.001049,0.8,1,-13.826698,-12.676280,-11.226344,-12.697430,-16.290809,-13.894953,-12.770068,-11.286722,-13.001177,-12.236377,-17.214381,-13.626404,-13.646623,-12.113496,-11.424510,-12.847130,-11.650951,-17.719171,-13.341640,1.872014,4


Los Angeles (SoCAB) 0.7417


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_l1_ratio,param_alpha,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,split10_test_score,split11_test_score,split12_test_score,split13_test_score,split14_test_score,mean_test_score,std_test_score,rank_test_score
3,0.023136,0.001613,0.006099,0.000178,0.1,0.3,-12.605386,-10.481693,-10.168905,-14.790110,-12.825984,-11.291090,-10.322671,-9.033640,-15.932126,-13.055285,-11.121742,-10.718685,-9.999178,-9.385906,-17.230539,-11.930863,2.362469,1
2,0.022297,0.002001,0.006487,0.001272,0.01,1,-12.700108,-11.016564,-10.504499,-14.859480,-12.897606,-11.740391,-10.540574,-9.237878,-15.987051,-13.158888,-11.427847,-11.328506,-10.197862,-9.600431,-17.201097,-12.159919,2.253798,2
0,0.040752,0.004404,0.006192,0.000142,0.8,0.03,-14.430257,-10.497231,-11.526007,-15.714818,-14.679148,-11.688246,-10.770553,-8.650573,-17.079468,-14.439929,-12.349381,-9.720864,-11.301546,-8.913641,-15.339716,-12.473425,2.543884,3
4,0.021511,0.001905,0.006075,0.000240,0.03,3,-12.929533,-11.377628,-11.659821,-15.579594,-13.058879,-12.076891,-10.957720,-9.654616,-16.764188,-13.348095,-11.827644,-11.497231,-10.945755,-10.046870,-17.996571,-12.648069,2.327765,4
1,0.132800,0.010079,0.004362,0.000867,0.9,0.01,-16.320280,-11.712339,-15.022608,-15.985888,-16.386930,-13.079448,-11.068072,-8.179614,-17.442357,-16.501286,-13.570348,-9.797681,-13.433686,-8.508559,-18.783844,-13.719529,3.191203,5


Los Angeles (SoCAB) 0.7739


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_l1_ratio,param_alpha,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,mean_test_score,std_test_score,rank_test_score
1,0.024989,0.002979,0.006532,0.000233,0.8,0.1,-10.535185,-10.725664,-12.444207,-10.653698,-10.256700,-12.285416,-11.462979,-11.194836,0.815071,1
0,0.037202,0.003068,0.006871,0.000279,0.2,0.03,-10.767922,-10.827195,-12.379878,-10.758208,-10.441481,-12.159763,-11.451348,-11.255114,0.702178,2
3,0.021440,0.002819,0.006415,0.000173,0.2,0.3,-10.820578,-10.577782,-12.808734,-10.959355,-10.423470,-12.597092,-11.787637,-11.424950,0.904198,3
5,0.094942,0.009132,0.006726,0.000129,0.03,0.01,-11.172721,-11.481927,-12.454378,-11.010956,-10.592624,-12.045633,-11.545752,-11.471999,0.582494,4
4,0.121416,0.010480,0.005649,0.001164,0.9,0.01,-11.571294,-12.262759,-12.322795,-11.212335,-10.801172,-12.191190,-11.701053,-11.723228,0.535102,5
2,0.020909,0.003018,0.006312,0.000236,0.01,1,-11.339761,-10.908084,-13.294444,-11.425958,-10.722353,-12.978442,-12.246692,-11.845105,0.934723,6


Los Angeles (SoCAB) 0.7802


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_l1_ratio,param_alpha,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,split10_test_score,mean_test_score,std_test_score,rank_test_score
0,0.041511,0.004856,0.007343,0.001571,0.1,0.1,-10.891704,-10.508358,-12.129457,-9.831004,-12.506519,-11.058132,-10.125997,-12.369324,-10.957888,-9.329873,-13.082904,-11.162833,1.153867,1
2,0.041429,0.002349,0.006795,0.000095,0.2,0.1,-10.895192,-10.507617,-12.110607,-9.833979,-12.546482,-11.060455,-10.127276,-12.376555,-10.941401,-9.333996,-13.125466,-11.169002,1.163096,2
1,0.038312,0.001915,0.006830,0.000132,0.2,0.3,-11.220737,-10.951053,-12.339470,-9.964279,-13.212594,-11.374553,-10.532211,-12.754759,-11.051117,-9.638642,-13.957173,-11.545144,1.299778,3
3,0.037319,0.002394,0.006872,0.000204,0.1,3,-12.235868,-12.060898,-13.540531,-10.950937,-14.342379,-12.438322,-11.349423,-14.192458,-11.973453,-10.897032,-15.044297,-12.638691,1.363148,4


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_l1_ratio,param_alpha,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,split10_test_score,split11_test_score,mean_test_score,std_test_score,rank_test_score
0,0.039612,0.003959,0.006921,0.000257,0.03,0.1,-10.395815,-11.524413,-11.571451,-10.641410,-10.923681,-11.005595,-11.494200,-10.837057,-9.855641,-12.261042,-10.132306,-12.083309,-11.060493,0.716133,1
2,0.038453,0.002811,0.007074,0.000415,0.01,0.3,-10.713761,-11.761680,-11.795274,-10.948107,-11.319264,-11.134869,-12.041657,-11.124718,-10.224065,-12.535457,-10.223806,-12.690537,-11.376100,0.776780,2
4,0.035586,0.002577,0.007097,0.000288,0.2,0.3,-10.758720,-11.739034,-11.723767,-11.003458,-11.367194,-11.091044,-12.190037,-11.184787,-10.226985,-12.503148,-10.221016,-12.929647,-11.411570,0.810163,3
3,0.035775,0.003066,0.007017,0.000268,0.01,1,-11.189136,-12.180064,-11.962198,-11.422740,-11.875336,-11.415727,-12.448271,-11.584385,-10.648081,-13.036281,-10.478905,-13.122032,-11.780263,0.798437,4
1,0.039110,0.007159,0.008379,0.002881,0.1,3,-11.699651,-12.921927,-12.773121,-11.960603,-12.732011,-12.007609,-13.328272,-12.177490,-11.094850,-13.955719,-11.123676,-14.021749,-12.483056,0.938552,5


In[ ]:

In[ ]:

In[ ]:

In [114]:
os.makedirs('clfs_{}'.format(dataset), exist_ok=True)

In[ ]:

In [115]:
pickle.dump((all_y_enet, all_y_pred_enet, enet_clfs, enet_scalers),
            open('clfs_{}/enet_clfs_{}.pkl'.format(
                 dataset, run_label), 'wb'))

for i in range(3):<br>
    pd.Series(np.mean([e.coef_ for e in<br>
                   enet_clfs[len(enet_clfs) * i//3:len(enet_clfs) * (i + 1)//3]],<br>
        axis = 0), index = x.columns[2:]).sort_values().plot(kind = 'barh',<br>
                                                                figsize = (10, x.shape[1]// 4),<br>
                                                                title = list(x.location.unique())[i], )<br>
    plt.figure()

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

print(round(np.corrcoef( pd.concat(all_y_pred_enet).clip(0, None),#.reindex(df.index),<br>
                            pd.concat(all_y_enet)#.reindex(df.index)<br>
           )[0, 1], 3))

In[ ]:

In [116]:
cs = []
for location, df in x.groupby('location'):
    c = np.corrcoef(pd.concat(all_y_pred_enet).clip(0, None).reindex(df.index),
                    pd.concat(all_y_enet).reindex(df.index))[0, 1]
    cs.append(c)
    print(location, round(c, 3))
print('\nBlend:', round(np.mean(cs), 3))

Los Angeles (SoCAB) 0.775

Blend: 0.775


In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [117]:
lgb_params = {
    # [ 150, 200, 300, ],
    'n_estimators': np.arange(200, 400, 10) if dataset == 'pm' else np.arange(300, 600, 20),
    # [0.03, 0.05, 0.07, ],
    'learning_rate': np.arange(0.01, 0.04, 0.003) if dataset == 'pm' else np.arange(0.01, 0.061, 0.005),
    # [5, 7, 10, 15, 20,],
    'num_leaves': np.arange(4, 30) if dataset == 'pm' else np.arange(10, 30),
    'min_child_weight': np.arange(0.02, 0.1, 0.01),  # [  0.1, 0.2, ],
    'min_child_samples': [140, 170, 200, 250, 300, 400, 500, 600, 700, 850, 1000, 1400, ]
    if dataset == 'pm' else
                         [30, 40, 50, 60, 80, 100, 120, 150,
                             170, 200, 300, 500, 700, ],
    'reg_lambda': [0, 1e-5, 1e-4, 1e-3, 1e-2, 0.1,],
    'reg_alpha': [0, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, ],
    'linear_tree': [True, ],
    'subsample': np.arange(0.4, 0.901, 0.05),  # [0.3, 0.5,  0.8],
    'subsample_freq': [1],
    # [0.5, 0.8, ],#0.3, 0.5, 0.8],
    'colsample_bytree': np.arange(0.3, 0.71, 0.05) if dataset == 'pm' else np.arange(0.2, 0.81, 0.05),
    # [0.5, 0.8, ],#0.3, 0.5, 0.8],
    'colsample_bynode': np.arange(0.4, 1.01, 0.05) if dataset == 'pm' else np.arange(0.2, 1.01, 0.05),
    'linear_lambda': [1e-3, 3e-3, 1e-2, 3e-2, 0.1,],
    # 'max_bins': np.arange(120, 400, 20),
    # 'min_data_in_bin': np.exp(np.arange(np.log(3), np.log(12), 0.1)).astype(int), #np.arange(2, 10),# [2, 3, 4, 5, 10],
    # [ 192,
    # 255, 255, 384, 512],
    'min_data_per_group': [10, 20, 50, 100],
    'max_cat_threshold': [8, 16, 32, ],
    'cat_l2': [0.1, 1, 10],  # if dataset == 'pm' else [1e-3, 1e-2, 1e-1],
    'cat_smooth': [0.1, 1, 10],  # if dataset == 'pm' else [1e-3, 1e-2, 1e-1],
}

In[ ]:

In [118]:
FAST = False
GAUSSIAN = 0.05  # if dataset == 'pm' else 0.1
COLUMN_WIPE = len(x.columns) // 5  # if dataset == 'pm' else 10)

In[ ]:

In[ ]:

In[ ]:

In [119]:
def runLGB(drop_cols=[], verbose=1, n_bags=8, ):
    all_y_lgb = []
    all_y_pred_lgb = []
    tidx = 0
    lgb_clfs = []
    lgb_scalers = []
    for location in x.location.unique():
        random.seed(datetime.datetime.now().microsecond)
        np.random.seed(datetime.datetime.now().microsecond)
        if FAST and location != 'Delhi':
            continue
        if verbose > 0:
            print(location)
        x_loc = x[x.location == location].drop(
            columns=drop_cols)  # .iloc[:, :4]
        y_loc = y.reindex(x_loc.index)
        d_loc = d.reindex(x_loc.index)
        folds = list(
            (PurgedKFold(4) if dataset == 'pm' else PurgedKFold(
                3, gap=20)) .split(
                x_loc, y_loc, d_loc))
        folds += [(np.arange(0, len(x_loc)), [])] * 3
        for train_fold, test_fold in folds:
            y_preds = []
            for i in range(n_bags):  # if location == 'Delhi' else 4):
                model = RandomizedSearchCV(lgb.LGBMRegressor(seed=datetime.datetime.now().microsecond,
                                                             # n_jobs = 2, #
                                                             # os.cpu_count()
                                                             ), lgb_params,
                                           cv=RepeatedPurgedKFold(random.randrange(3, 6),
                                                                  n_repeats=random.randrange(
                                                                      1, 3),
                                                                  gap=random.randrange(60, 120)),
                                           n_iter=random.randrange(3, 5),
                                           scoring=SCORING,
                                           n_jobs=-1,  # os.cpu_count(),
                                           random_state=datetime.datetime.now().microsecond)
                l = random.randrange(0, len(train_fold) // 10)
                s = random.randrange(0, len(train_fold) - l)
                subset = train_fold[:s].tolist() + train_fold[s + l:].tolist()
                xt = x_loc.iloc[subset].copy()
                xt.iloc[:, 2:] += (GAUSSIAN * random.random() *
                                   np.random.default_rng().standard_normal(size=xt.iloc[:, 2:].shape)
                                   * xt.iloc[:, 2:].std().values[None, :])
                for c in random.choices(xt.columns[2:], k=int(
                        round(random.random() * COLUMN_WIPE))):
                    xt[c] = 0
                if dataset == 'pm' and random.random() < 0.3:
                    xt['dayinyear'] = 0
                sample_cols = [
                    c for c in xt.columns if any(
                        z in c for z in [
                            'precision',
                            'air_mass',
                            'stdev'] + [
                            'no2',
                            'so2',
                            'co',
                            'o3',
                            'pm25_rh35_gcc',
                            'ifs'])]
                xt[random.sample(sample_cols, k=random.randrange(
                    len(sample_cols) * 1 // 3, len(sample_cols) * 2 // 3))] = 0
                # if random.random() < {'Delhi': 0.7, 'Taipei': 0.2}.get(location, 0):
                #     xt[[c for c in xt.columns if 'FineMode' in c or 'Injection' in c]] = 0
                # if random.random() < {'Delhi': 0.1, 'Taipei': 0.1}.get(location, 0):
                #     xt[[c for c in xt.columns if 'Optical' in c]] = 0
                # if random.random() < {'Delhi': 1.0, 'Taipei': 0.2}.get(location, 0.2):
                #     xt[[c for c in xt.columns if c.startswith('ifs')]] = 0
                # if random.random() < {'Delhi': 0.9,}.get(location, 0.12):
                #     xt[[c for c in xt.columns if any(z + '_' in c for z in
                #          ['no2', 'so2', 'co', 'o3', 'pm25_rh35_gcc',  ] ) ]] = 0
                scaler = StandardScaler()
                xt.iloc[:, 2:] = scaler.fit_transform(
                    xt.iloc[:, 2:]).astype(np.float32)
                # for i in xt, y_loc.iloc[subset], d_loc.iloc[subset]:
                #     if i
                y_loc_subset = y_loc.iloc[subset].astype(np.int64)
                d_loc_subset = d_loc.iloc[subset].astype(np.int64) # //1e9
                print(f'xt = {xt}')
                print(f'y_loc.iloc[subset] = {y_loc_subset}')
                print(f'd_loc.iloc[subset] = {d_loc_subset}')
                model.fit(xt, y_loc_subset, groups=d_loc_subset,
                         )
                lgb_clfs.append(model.best_estimator_)
                lgb_scalers.append(scaler)
                if len(test_fold) > 0:
                    xtst = x_loc.iloc[test_fold].copy()
                    xtst.iloc[:, 2:] = scaler.transform(
                        xtst.iloc[:, 2:]).astype(np.float32)
                    y_pred = pd.Series(
                        model.predict(xtst),
                        index=y_loc.iloc[test_fold].index)
                    y_preds.append(y_pred)
                df = pd.DataFrame(model.cv_results_).sort_values(
                    'rank_test_score').drop(columns='params')
                if i == 0 and verbose > 1:  # df.mean_test_score.min() < -0:
                    display(df)
                    print()
            if len(test_fold) > 0:
                y_pred = pd.concat(y_preds)
                y_pred = (y_pred.groupby(y_pred.index).mean())  # * 2/3
                # + y_pred.groupby(y_pred.index).median() * 1/3 )
                all_y_lgb.append(y_loc.iloc[test_fold])
                all_y_pred_lgb.append(y_pred)
                if verbose >= 3:
                    all_y_lgb[-1].reset_index(
                        drop=True).ewm(span=10).mean().plot()
                    plt.plot(y_pred.reset_index(drop=True), linewidth=0.8)
                    try:
                        plt.plot(
                            all_y_pred_enet[tidx].clip(
                                0, None).reset_index(
                                drop=True))
                    except BaseException:
                        pass
                    plt.title(location)
                    plt.figure()
                tidx += 1
                if verbose > 0:
                    print(location, round(np.corrcoef(
                        y_pred, all_y_lgb[-1])[0, 1], 4))
    return all_y_lgb, all_y_pred_lgb, lgb_clfs, lgb_scalers

In[ ]:

In [120]:
all_y_lgb, all_y_pred_lgb, lgb_clfs, lgb_scalers = runLGB(verbose=2, )

Los Angeles (SoCAB)
xt =       grid_id             location  dayofweek  dayinyear  \
6307    3A3IE  Los Angeles (SoCAB)  -1.496396          0   
6308    3S31A  Los Angeles (SoCAB)  -1.479956          0   
6309    7II4T  Los Angeles (SoCAB)  -1.490762          0   
6310    8BOQH  Los Angeles (SoCAB)  -1.497700          0   
6311    A2FBI  Los Angeles (SoCAB)  -1.504538          0   
...       ...                  ...        ...        ...   
17468   WT52R  Los Angeles (SoCAB)   1.019184          0   
17469   X5DKW  Los Angeles (SoCAB)   1.027057          0   
17470   Z0VWC  Los Angeles (SoCAB)   1.017141          0   
17471   ZP1FZ  Los Angeles (SoCAB)   1.015855          0   
17472   ZZ8JF  Los Angeles (SoCAB)   1.031667          0   

       Wind speed (gust)_local_3day  Wind speed (gust)_city_3day  \
6307                      -0.790051                    -0.528354   
6308                      -0.571249                    -0.545501   
6309                      -0.686957               

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow 

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample_freq,param_subsample,param_reg_lambda,param_reg_alpha,param_num_leaves,param_n_estimators,param_min_data_per_group,param_min_child_weight,param_min_child_samples,param_max_cat_threshold,param_linear_tree,param_linear_lambda,param_learning_rate,param_colsample_bytree,param_colsample_bynode,param_cat_smooth,param_cat_l2,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,mean_test_score,std_test_score,rank_test_score
0,5.522057,0.501759,0.239269,0.024555,1,0.7,0.01,0.001,11,560,20,0.02,150,8,True,0.001,0.015,0.75,0.45,10,10,-9.076244,-9.911945,-8.063718,-9.752136,-8.463935,-11.187346,-7.830947,-7.076110,-9.929798,-9.032464,1.214483,1
2,1.959985,0.349190,0.176190,0.034253,1,0.45,0,0,16,460,10,0.03,150,16,True,0.003,0.02,0.3,0.6,10,0.1,-9.345498,-9.882361,-7.612690,-9.853007,-8.851705,-11.223417,-8.147141,-6.687056,-10.447821,-9.116744,1.356726,2
3,1.520220,0.341376,0.103900,0.024778,1,0.8,0,0.01,26,440,50,0.05,500,8,True,0.01,0.05,0.35,1.0,10,10,-9.160746,-9.956492,-8.718799,-10.705928,-8.716517,-11.005772,-8.256107,-7.004825,-10.618759,-9.349327,1.251207,3
1,1.592918,0.321141,0.100931,0.008375,1,0.65,0.0001,0.1,28,400,100,0.06,500,8,True,0.01,0.05,0.5,0.2,10,0.1,-9.534680,-10.193051,-8.251505,-11.060607,-9.192806,-11.610818,-8.444851,-7.677092,-10.563621,-9.614337,1.268229,4



xt =       grid_id             location  dayofweek  dayinyear  \
6307    3A3IE  Los Angeles (SoCAB)  -1.510176   0.418851   
6308    3S31A  Los Angeles (SoCAB)  -1.492052   0.423514   
6309    7II4T  Los Angeles (SoCAB)  -1.488804   0.421807   
6310    8BOQH  Los Angeles (SoCAB)  -1.478754   0.436193   
6311    A2FBI  Los Angeles (SoCAB)  -1.494104   0.419872   
...       ...                  ...        ...        ...   
17468   WT52R  Los Angeles (SoCAB)   1.006175   1.027243   
17469   X5DKW  Los Angeles (SoCAB)   1.020725   1.028327   
17470   Z0VWC  Los Angeles (SoCAB)   1.017152   1.043053   
17471   ZP1FZ  Los Angeles (SoCAB)   1.016857   1.030253   
17472   ZZ8JF  Los Angeles (SoCAB)   0.999132   1.046229   

       Wind speed (gust)_local_3day  Wind speed (gust)_city_3day  \
6307                      -0.787908                    -0.550864   
6308                      -0.574107                    -0.561351   
6309                      -0.695667                    -0.559736   
6

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample_freq,param_subsample,param_reg_lambda,param_reg_alpha,param_num_leaves,param_n_estimators,param_min_data_per_group,param_min_child_weight,param_min_child_samples,param_max_cat_threshold,param_linear_tree,param_linear_lambda,param_learning_rate,param_colsample_bytree,param_colsample_bynode,param_cat_smooth,param_cat_l2,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,mean_test_score,std_test_score,rank_test_score
2,3.115026,0.564548,0.139490,0.042540,1,0.65,0.01,0,17,460,20,0.04,200,8,True,0.1,0.035,0.65,0.7,0.1,0.1,-9.565930,-6.081501,-8.074775,-9.838235,-9.735097,-6.684043,-7.295270,-5.867250,-12.026428,-8.352059,1.955098,1
0,4.748026,0.401480,0.219608,0.046292,1,0.55,0,0.00001,19,440,100,0.07,80,8,True,0.01,0.055,0.55,0.5,1,0.1,-9.974890,-5.989894,-7.381404,-9.214125,-10.704747,-7.108507,-7.683920,-6.003326,-11.290802,-8.372402,1.873076,2
1,1.990385,0.133335,0.093766,0.011809,1,0.85,0.01,0.0001,12,320,10,0.06,50,16,True,0.003,0.015,0.3,0.4,1,1,-9.580556,-7.058159,-7.498576,-10.466443,-10.923466,-7.493263,-7.878213,-6.086786,-11.764435,-8.749989,1.865097,3


...


xt =       grid_id             location  dayofweek  dayinyear  \
0       3A3IE  Los Angeles (SoCAB)  -1.003111  -1.678174   
1       3S31A  Los Angeles (SoCAB)  -0.995249  -1.678516   
2       7II4T  Los Angeles (SoCAB)  -1.003212  -1.677014   
3       8BOQH  Los Angeles (SoCAB)  -1.002040  -1.679763   
4       A2FBI  Los Angeles (SoCAB)  -0.996356  -1.677661   
...       ...                  ...        ...        ...   
17468   WT52R  Los Angeles (SoCAB)   1.003853   1.391871   
17469   X5DKW  Los Angeles (SoCAB)   1.016942   1.400254   
17470   Z0VWC  Los Angeles (SoCAB)   1.006924   1.400900   
17471   ZP1FZ  Los Angeles (SoCAB)   1.008918   1.399085   
17472   ZZ8JF  Los Angeles (SoCAB)   1.010056   1.404758   

       Wind speed (gust)_local_3day  Wind speed (gust)_city_3day  \
0                          2.733074                     2.768556   
1                          3.312851                     2.772851   
2                          1.059300                     2.782063   
3

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample_freq,param_subsample,param_reg_lambda,param_reg_alpha,param_num_leaves,param_n_estimators,param_min_data_per_group,param_min_child_weight,param_min_child_samples,param_max_cat_threshold,param_linear_tree,param_linear_lambda,param_learning_rate,param_colsample_bytree,param_colsample_bynode,param_cat_smooth,param_cat_l2,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,10.229186,0.601256,0.577762,0.009241,1,0.9,0.001,0.1,29,520,50,0.08,150,32,True,0.01,0.04,0.45,1.0,0.1,10,-7.157618,-8.418055,-8.513430,-8.029701,0.617884,1
3,5.231213,0.882590,0.308597,0.060893,1,0.6,0.001,0.0001,17,380,100,0.09,150,32,True,0.001,0.04,0.5,0.4,10,0.1,-7.212254,-8.426826,-9.160262,-8.266447,0.803316,2
1,2.381955,0.403906,0.304620,0.063114,1,0.6,0.1,0.00001,12,420,10,0.07,700,16,True,0.003,0.04,0.35,0.6,1,0.1,-7.364858,-8.931804,-8.789284,-8.361982,0.707470,3
2,6.420540,0.358094,0.339879,0.010556,1,0.4,0.00001,0.0001,25,320,50,0.07,30,32,True,0.01,0.03,0.45,0.2,0.1,10,-7.668685,-8.634938,-9.192639,-8.498754,0.629560,4



xt =       grid_id             location  dayofweek  dayinyear  \
0       3A3IE  Los Angeles (SoCAB)  -0.998105  -1.705777   
1       3S31A  Los Angeles (SoCAB)  -0.990667  -1.670491   
2       7II4T  Los Angeles (SoCAB)  -0.989792  -1.721506   
3       8BOQH  Los Angeles (SoCAB)  -1.036496  -1.715777   
4       A2FBI  Los Angeles (SoCAB)  -1.009567  -1.678143   
...       ...                  ...        ...        ...   
17468   WT52R  Los Angeles (SoCAB)   1.070105   1.401489   
17469   X5DKW  Los Angeles (SoCAB)   1.006758   1.375312   
17470   Z0VWC  Los Angeles (SoCAB)   0.996040   1.430336   
17471   ZP1FZ  Los Angeles (SoCAB)   1.021393   1.360901   
17472   ZZ8JF  Los Angeles (SoCAB)   1.004935   1.362390   

       Wind speed (gust)_local_3day  Wind speed (gust)_city_3day  \
0                          2.760455                     2.777970   
1                          3.312279                     2.779388   
2                          1.055045                     2.798711   
3

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample_freq,param_subsample,param_reg_lambda,param_reg_alpha,param_num_leaves,param_n_estimators,param_min_data_per_group,param_min_child_weight,param_min_child_samples,param_max_cat_threshold,param_linear_tree,param_linear_lambda,param_learning_rate,param_colsample_bytree,param_colsample_bynode,param_cat_smooth,param_cat_l2,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,8.982360,0.348188,0.474587,0.055803,1,0.55,0.01,0.001,21,560,10,0.02,150,16,True,0.1,0.055,0.55,0.35,10,10,-7.074026,-6.626449,-8.684137,-6.886636,-7.896924,-7.433635,0.756104,1
2,4.138436,0.865720,0.191278,0.043552,1,0.8,0.0001,0.0001,18,500,100,0.02,700,8,True,0.003,0.055,0.6,0.7,10,10,-7.070337,-6.661282,-8.957760,-7.157574,-7.863110,-7.542012,0.806648,2
1,5.145291,0.233681,0.233183,0.034954,1,0.4,0.01,0.0001,10,480,20,0.05,120,8,True,0.003,0.06,0.65,0.6,0.1,10,-7.201938,-6.579456,-8.743982,-7.125258,-8.983039,-7.726734,0.955670,3



xt =       grid_id             location  dayofweek  dayinyear  \
0       3A3IE  Los Angeles (SoCAB)          0  -1.733094   
1       3S31A  Los Angeles (SoCAB)          0  -1.750883   
2       7II4T  Los Angeles (SoCAB)          0  -1.734352   
3       8BOQH  Los Angeles (SoCAB)          0  -1.726911   
4       A2FBI  Los Angeles (SoCAB)          0  -1.714833   
...       ...                  ...        ...        ...   
17468   WT52R  Los Angeles (SoCAB)          0   1.297983   
17469   X5DKW  Los Angeles (SoCAB)          0   1.300664   
17470   Z0VWC  Los Angeles (SoCAB)          0   1.289595   
17471   ZP1FZ  Los Angeles (SoCAB)          0   1.288578   
17472   ZZ8JF  Los Angeles (SoCAB)          0   1.301422   

       Wind speed (gust)_local_3day  Wind speed (gust)_city_3day  \
0                          2.759830                     2.898162   
1                          3.305822                     2.862613   
2                          1.056070                     2.863848   
3

In[ ]:

In [121]:
pickle.dump((all_y_lgb, all_y_pred_lgb, lgb_clfs, lgb_scalers),
            open('clfs_{}/lgb_clfs_{}.pkl'.format(
                 dataset, run_label), 'wb'))

In[ ]:

In[ ]:

# print(round(np.corrcoef( pd.concat(all_y_pred_enet).clip(0, None), pd.concat(all_y_lgb))[0, 1], 3)  )<br>
print(round(np.corrcoef( pd.concat(all_y_pred_lgb).clip(0, None), pd.concat(all_y_lgb))[0, 1], 3) )<br>
print(round(np.corrcoef( pd.concat(all_y_pred_lgb).clip(0, None)<br>
                  +  0.1  * pd.concat(all_y_pred_enet).clip(0, None)<br>
                  , pd.concat(all_y_lgb))[0, 1], 4) )<br>
 # without weather ~0.79 lgb

In[ ]:

In [122]:
cs = []
for location, df in x.groupby('location'):
    c = np.corrcoef(
        pd.concat(all_y_pred_lgb).clip(
            0,
            None).reindex(
            df.index) +
        0.1 *
        pd.concat(all_y_pred_enet).clip(
            0,
            None).reindex(
            df.index),
        pd.concat(all_y_lgb).reindex(
            df.index))[
        0,
        1]
    cs.append(c)
    print(location,
          round(c, 3))
print('\nBlend: ', round(np.mean(cs), 3))

Los Angeles (SoCAB) 0.913

Blend:  0.913


In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [123]:
len(lgb_clfs)

48

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [124]:
submission = pd.read_csv('data_{}/submission_format.csv'.format(dataset))

In[ ]:

In [125]:
submission['dayinyear'] = pd.to_datetime(submission.datetime).dt.dayofyear
submission['dayofweek'] = pd.to_datetime(submission.datetime).dt.dayofweek

In[ ]:

In [126]:
submission['location'] = grid.set_index(
    'grid_id').location.reindex(submission.grid_id).values
submission['location'] = submission.location.astype('category')
submission['grid_id'] = submission.grid_id.astype(
    x.grid_id.dtype)  # 'category')

In[ ]:

In [127]:
submission = addGFS(submission)
submission = addIFS(submission)
submission = addSat(submission, 'maiac' if dataset == 'pm' else tropomi)
if ASSIM:
    submission = addSat(submission, assim)

In[ ]:

In[ ]:

In[ ]:

In [128]:
pickle.dump(submission, open('cache/submission_{}.pkl'.format(dataset), 'wb'))

In[ ]:

In[ ]:

In[ ]:

In [129]:
xs = submission[x.columns]

In[ ]:

In [130]:
submission.location.value_counts()

location
Los Angeles (SoCAB)    9039
Name: count, dtype: int64

In[ ]:

In[ ]:

In[ ]:

In [131]:
clf_path = 'clfs_{}'.format(dataset)
all_clfs = os.listdir(clf_path)

enet_tuples = [pickle.load(open( os.path.join(clf_path, f), 'rb'))<br>
                   for f in all_clfs if 'enet_clfs' in f] if dataset == 'pm' else []<br>
lgb_tuples = [pickle.load(open( os.path.join(clf_path, f), 'rb'))<br>
                   for f in all_clfs if 'lgb_clfs' in f]

In[ ]:

In [132]:
enet_tuples = [(all_y_enet, all_y_pred_enet, enet_clfs, enet_scalers)]
lgb_tuples = [(all_y_lgb, all_y_pred_lgb, lgb_clfs, lgb_scalers)]

In[ ]:

In [133]:
len(lgb_tuples)

1

In[ ]:

In [134]:
len(enet_tuples)

1

labels

In[ ]:

In [135]:
y_true = pd.concat(flatten([e[0] for e in lgb_tuples]))
y_true = y_true.groupby(y_true.index).mean()

In[ ]:

In [136]:
y_pred_lgb = pd.concat(flatten([e[1] for e in lgb_tuples]))
y_pred_lgb = y_pred_lgb.groupby(y_pred_lgb.index).mean()  # * 2/3
# + y_pred_lgb.groupby(y_pred_lgb.index).median() * 1/3)

In[ ]:

In [137]:
if dataset == 'pm':
    y_pred_enet = pd.concat(flatten([e[1] for e in enet_tuples]))
    y_pred_enet = (y_pred_enet.groupby(y_pred_enet.index).mean())
    # + y_pred_enet.groupby(y_pred_enet.index).mean() * 1/3)

In[ ]:

In [138]:
cs = []
for location, df in x.groupby('location'):
    c = np.corrcoef(
        y_pred_lgb.clip(
            0, None).reindex(
            df.index) + 1 / 5 * (
                y_pred_enet.clip(
                    0, None).reindex(
                        df.index) if dataset == 'pm' else 0), y_true.reindex(
                            df.index))[
                                0, 1]
    cs.append(c)
    print(location,
          round(c, 3))
print('\nBlend: ', round(np.mean(cs), 3))

Los Angeles (SoCAB) 0.913

Blend:  0.913


In[ ]:

In[ ]:

In[ ]:

[c.n_features_in_ for c in enet_scalers]

In[ ]:

In [146]:
lgb_clfs, lgb_scalers, enet_clfs, enet_scalers = [], [], [], []
for i in range(3):
    lgb_clfs.extend(flatten([e[2][i * len(e[2]) // 3: (i + 1) * len(e[2]) // 3]
                             for e in lgb_tuples]))
    lgb_scalers.extend(flatten(
        [e[3][i * len(e[3]) // 3: (i + 1) * len(e[3]) // 3] for e in lgb_tuples]))
    enet_clfs.extend(flatten([e[2][i * len(e[2]) // 3: (i + 1) * len(e[2]) // 3]
                              for e in enet_tuples]))
    enet_scalers.extend(flatten(
        [e[3][i * len(e[3]) // 3: (i + 1) * len(e[3]) // 3] for e in enet_tuples]))

In[ ]:

In[ ]:

In[ ]:

In [147]:
len(lgb_clfs)

48

In[ ]:

In [152]:
lgb_ys = []
for clf_idx, clf in enumerate(lgb_clfs):
    x_loc = xs[xs.location == x.location.unique()[0]].copy()  # Always use the first (and only) location
    x_loc.iloc[:, 2:] = lgb_scalers[clf_idx].transform(x_loc.iloc[:, 2:])
    lgb_ys.append(pd.Series(clf.predict(x_loc), index=x_loc.index))
    lgb_y_combined = pd.concat(lgb_ys, axis=0)

In [153]:
enet_ys = []
for clf_idx, clf in enumerate(enet_clfs):
    x_loc = xs[xs.location == x.location.unique()[0]].copy().iloc[:,2:]
    enet_ys.append(pd.Series(clf.predict(
        pd.DataFrame(enet_scalers[clf_idx].transform(x_loc), 
                     columns = x_loc.columns)), index = x_loc.index))
enet_ys = pd.concat(enet_ys)

lgb_ys.groupby(lgb_ys.index).std().plot()

In [155]:
lgb_y = lgb_y_combined.groupby(lgb_y_combined.index).mean().clip(0, None)  # .sort_values()

In [156]:
enet_y = enet_ys.groupby(enet_ys.index).mean().clip(0, None)  # .sort_values()

In [164]:
# if dataset == 'pm':<br>
# plt.scatter(enet_y, lgb_y, s= 0.1)<br>
# print(round(np.corrcoef(enet_y, lgb_y)[0,1], 4) )#, s= 0.1)

In [157]:
f = 8 if dataset == 'pm' else 100000  # lgb_y
ys = (lgb_y * (f - 1) / f + enet_y * 1 / f)

In [158]:
ys.name = 'value'

In [159]:
out = pd.concat((submission[['datetime', 'grid_id']],
                ys.reindex(submission.index)), axis=1)
out

,datetime,grid_id,value
0,2018-09-08T08:00:00Z,3A3IE,32.926282
1,2018-09-08T08:00:00Z,3S31A,18.227048
2,2018-09-08T08:00:00Z,7II4T,29.309733
3,2018-09-08T08:00:00Z,8BOQH,15.247705
4,2018-09-08T08:00:00Z,A2FBI,26.205926
...,...,...,...
9034,2021-08-24T08:00:00Z,WT52R,11.446385
9035,2021-08-24T08:00:00Z,X5DKW,48.123762
9036,2021-08-24T08:00:00Z,Z0VWC,14.724684
9037,2021-08-24T08:00:00Z,ZP1FZ,14.510786


In [ ]:
# out[pd.to_datetime(out.datetime) < datetime.datetime(2018, 1, 1)]

In [ ]:
# if dataset == 'pm' and ASSIM:<br>
#     pivot = '2018-01-10'<br>
#     b1 = pd.read_csv('submissions_pm/blend1.csv')<br>
#     mix = pd.concat((<br>
#         b1[b1.datetime < pivot],<br>
#         out[out.datetime >= pivot] ) )<br>
#     assert out.shape == mix.shape<br>
#     out = mix

In [ ]:
# for g in out.grid_id.unique():<br>
#     out[out.grid_id == g].set_index(pd.to_datetime(out[out.grid_id == g].datetime)).value.plot(<br>
#         marker = '.', linewidth = 0.1,)# markersize = 3)<br>
# plt.xlim(pd.to_datetime(submission.datetime).min(),<br>
# pd.to_datetime(submission.datetime).min() + datetime.timedelta(days =<br>
# 400));

In [162]:
# for g in out.grid_id.unique():<br>
#     out[out.grid_id == g].set_index(pd.to_datetime(out[out.grid_id == g].datetime)).value.plot(<br>
#         marker = '.', linewidth = 0.1,)# markersize = 3)<br>
# plt.xlim(datetime.datetime(2020, 10, 15), datetime.datetime(2021, 9, 1));

In [ ]:
# if dataset == 'pm':<br>
#     sp = pd.read_csv('submissions_pm/lgb_baseline.csv')<br>
#     sp2 = pd.read_csv('submissions_pm/lgb_2.csv')<br>
#     g1 = pd.read_csv('submissions_pm/first_gfs.csv')<br>
#     m1 = pd.read_csv('submissions_pm/maiac.csv')<br>
#     m2 = pd.read_csv('submissions_pm/maiac2.csv')<br>
#     m3 = pd.read_csv('submissions_pm/maiac3.csv')<br>
#     b1 = pd.read_csv('submissions_pm/blend1.csv')<br>
#     b1f = pd.read_csv('submissions_pm/blend1f.csv')<br>
# else:<br>
#     sp = pd.read_csv('../submissions_tg/third_gfs.csv')<br>
#     s1 = pd.read_csv('../submissions_tg/sat1.csv')<br>
#     s2 = pd.read_csv('../submissions_tg/sat2.csv')<br>
#     s4 = pd.read_csv('../submissions_tg/sat4.csv')<br>
#     a1 = pd.read_csv('../submissions_tg/assim1.csv')<br>
#     n = pd.read_csv('../submissions_tg/new.csv')<br>
#     st1 = pd.read_csv('../submissions_tg/stack1.csv')<br>
#     n2 = pd.read_csv('submissions_tg/new.csv')

In [ ]:
# if dataset == 'pm':<br>
#     # plt.scatter(sp.value, out.value, s= 0.1);<br>
#     # plt.scatter(sp2.value, out.value, s= 0.1);<br>
#     # plt.scatter(g1.value, out.value, s= 0.1);<br>
#     plt.scatter(m1.value, out.value, s= 0.1);<br>
#     plt.scatter(m2.value, out.value, s= 0.1);<br>
#     plt.scatter(m3.value, out.value, s= 0.1);<br>
#     plt.scatter(b1.value, out.value, s= 0.1);<br>
#     plt.scatter(b1f.value, out.value, s= 0.1);<br>
# else:<br>
#     plt.scatter(sp.value, out.value, s= 0.1);<br>
#     plt.scatter(s1.value, out.value, s= 0.1);<br>
#     plt.scatter(s2.value, out.value, s= 0.1);<br>
#     plt.scatter(s4.value, out.value, s= 0.1);<br>
#     plt.scatter(a1.value, out.value, s= 0.1);<br>
#     plt.scatter(n.value, out.value, s= 0.1);<br>
#     plt.scatter(n2.value, out.value, s= 0.1)<br>
#     plt.scatter(st1.value, out.value, s= 0.1)

In [ ]:
# if dataset == 'pm':<br>
#     print(np.corrcoef((g1.value, m1.value, m2.value, m3.value, b1.value, b1f.value, out.value)).round(4))<br>
# else:<br>
#     print(np.corrcoef((sp.value, s1.value, s2.value, s4.value, a1.value, n.value, n2.value, st1.value, out.value)).round(4))

In [160]:
out[out.datetime < '2021-04-01']

,datetime,grid_id,value
0,2018-09-08T08:00:00Z,3A3IE,32.926282
1,2018-09-08T08:00:00Z,3S31A,18.227048
2,2018-09-08T08:00:00Z,7II4T,29.309733
3,2018-09-08T08:00:00Z,8BOQH,15.247705
4,2018-09-08T08:00:00Z,A2FBI,26.205926
...,...,...,...
5617,2021-03-31T08:00:00Z,WT52R,14.291925
5618,2021-03-31T08:00:00Z,X5DKW,51.693710
5619,2021-03-31T08:00:00Z,Z0VWC,31.615251
5620,2021-03-31T08:00:00Z,ZP1FZ,14.305387


In [161]:
os.makedirs('submissions_{}'.format(dataset), exist_ok=True)
out.to_csv('submissions_{}/new.csv'.format(dataset), index=False)